# Module B: Vector Ops & RBAC Security Console

**Goal:** Visualize "Meaning" as "Distance" and implement Security Filters.

**Persona:** AI Architect

**Hardware:** GPU (Optional but recommended for speed demo)


In [1]:
# -- SETUP --

import json
import os
import glob

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output

from sentence_transformers import SentenceTransformer

print("✅ Cockpit Initialized.")


✅ Cockpit Initialized.


## Phase 1: The Vector Radar (Core)

Visualize how the model clusters concepts. We have injected 3 distinct clusters.


In [2]:
# -- DATA LOADING --

# Synthetic Data representing FICO clusters
data = {
    'text': [
        # Cluster A: Finance
        "APR calculation method", "Credit score risk factor", "Loan default probability", 
        "Interest rate adjustment", "FICO Score 8 vs 9", "Debt to Income Ratio",
        # Cluster B: Infrastructure
        "Kubernetes pod crash", "GPU memory overflow", "Latency spike in US-East",
        "Firewall rule block", "Docker container restart", "NVIDIA driver update",
        # Cluster C: Noise (Food/General)
        "Pepperoni pizza recipe", "How to bake a cake", "Best spicy tacos",
        "Weather in London", "Football match results", "Organic apple pie"
    ],
    'cluster': ['Finance']*6 + ['Infrastructure']*6 + ['Noise']*6
}


df = pd.DataFrame(data)


# Generate pseudo-embeddings (3D) for visualization purposes
# In reality, these would be 1536-dim vectors flattened via PCA
np.random.seed(42)
# Finance centers around (1, 1, 0)
vecs_a = np.random.normal(loc=[1, 1, 0], scale=0.2, size=(6, 3))
# Infra centers around (-1, 1, 0)
vecs_b = np.random.normal(loc=[-1, 1, 0], scale=0.2, size=(6, 3))
# Noise centers around (0, -1, 0.5)
vecs_c = np.random.normal(loc=[0, -1, 0.5], scale=0.3, size=(6, 3))


vectors = np.vstack([vecs_a, vecs_b, vecs_c])
df['x'] = vectors[:,0]
df['y'] = vectors[:,1]
df['z'] = vectors[:,2]


print("✅ Data Vectorized. 3 Clusters Detected.")


✅ Data Vectorized. 3 Clusters Detected.


In [3]:
# -- INTERACTIVE RADAR --
# This cell renders the 3D Plot


def plot_radar(inject_query=None):
    fig = px.scatter_3d(
        df, x='x', y='y', z='z', color='cluster',
        symbol='cluster', opacity=0.8,
        title="FICO Vector Space Radar",
    )

    if inject_query:
        # Simple heuristic to place the injected query for the demo
        # "Server" -> Infra, "Credit" -> Finance, Else -> Void
        q_lower = inject_query.lower()
        if any(x in q_lower for x in ['server', 'crash', 'gpu', 'slow']):
            loc = [-1, 1, 0] + np.random.normal(0, 0.1, 3)  # Near Infra
        elif any(x in q_lower for x in ['credit', 'score', 'loan', 'rate']):
            loc = [1, 1, 0] + np.random.normal(0, 0.1, 3)  # Near Finance
        else:
            loc = [0, 0, 2]  # The "Void" (Hallucination space)

        fig.add_trace(
            go.Scatter3d(
                x=[loc[0]],
                y=[loc[1]],
                z=[loc[2]],
                mode='markers+text',
                marker=dict(size=15, color='red', symbol='x'),
                text=[f"QUERY: {inject_query}"],
                textposition="top center",
                name='Injected Query',
            )
        )

    fig.update_layout(margin=dict(l=0, r=0, b=0, t=30))
    fig.show()


# Widget Setup
text_input = widgets.Text(
    value='',
    placeholder='Type a query (e.g., "Server is crashing")',
    description='Query:',
)
button = widgets.Button(description="Inject Query", button_style='danger')


def on_click(b):
    clear_output(wait=True)
    display(ui)
    plot_radar(text_input.value)


button.on_click(on_click)
ui = widgets.VBox([text_input, button])


display(ui)
plot_radar()


## Phase 2: RBAC Security Filter (Extension)

**Scenario:** A "Junior Analyst" tries to access sensitive data.

**Task:** Filter search results based on `user_level`.


In [ ]:
# -- RBAC + REAL VECTOR SEARCH --
# 1) Embed the query
# 2) Retrieve top-K by cosine similarity over saved embeddings
# 3) Apply RBAC filter (tenant/role/restricted tags)

ROLE_LEVEL = {
    "public": 1,
    "analyst": 2,
    "risk_analyst": 2,
    "sre": 3,
    "security": 3,
    "admin": 3,
}

ALL_ROLES = list(ROLE_LEVEL.keys())


def rbac_allow(row, user_role: str, user_tenant: str, clearance_tags: set[str]):
    allowed_roles = row.get("allowed_roles", []) or []
    allowed_tenants = row.get("allowed_tenants", []) or []
    restricted_tags = row.get("restricted_tags", []) or []

    # Admin/security commonly have cross-tenant visibility.
    if user_role in {"admin", "security"}:
        tenant_ok = True
    else:
        tenant_ok = (
            "*" in allowed_tenants
            or (user_tenant in allowed_tenants)
            or (str(row.get("tenant_id", "global")) in ["global", user_tenant])
        )

    if "public" in allowed_roles:
        role_ok = True
    else:
        role_ok = user_role in allowed_roles

    if restricted_tags:
        if user_role in {"admin", "security"}:
            tags_ok = True
        else:
            tags_ok = set(restricted_tags).issubset(clearance_tags)
    else:
        tags_ok = True

    ok = tenant_ok and role_ok and tags_ok

    reason = []
    if not tenant_ok:
        reason.append("tenant")
    if not role_ok:
        reason.append("role")
    if not tags_ok:
        reason.append("restricted_tags")

    return ok, "+".join(reason) if reason else "ok"


def vector_search(query: str, top_k: int = 30):
    q = (query or "").strip()
    if not q:
        return [], None

    q_emb = embedder.encode([q], normalize_embeddings=True, convert_to_numpy=True, show_progress_bar=False)[0].astype(np.float32)
    q_norm = q_emb / (np.linalg.norm(q_emb) + 1e-12)

    sims = E_norm @ q_norm

    k = min(int(top_k), len(sims))
    idx = np.argpartition(-sims, k - 1)[:k]
    idx = idx[np.argsort(-sims[idx])]

    results = [(int(i), float(sims[int(i)])) for i in idx]
    return results, q_emb


def rbac_vector_search(query: str, user_role: str, user_tenant: str, clearance_tags: set[str], top_k: int = 30):
    q = (query or "").strip()
    if not q:
        print("❌ Enter a query.")
        return

    print(f"🔎 Semantic search: '{q}' as role='{user_role}' tenant='{user_tenant}' clearance={sorted(list(clearance_tags))}")

    hits, _ = vector_search(q, top_k=top_k)
    if not hits:
        print("❌ No results.")
        return

    visible = []
    blocked_reasons = {}

    for i, score in hits:
        row = df.iloc[i]
        ok, reason = rbac_allow(row, user_role, user_tenant, clearance_tags)
        if ok:
            visible.append((row, score))
        else:
            blocked_reasons[reason] = blocked_reasons.get(reason, 0) + 1

    print("----- Results -----")
    if visible:
        for row, score in visible[:20]:
            title = row.get("title", "(no title)")
            doc_type = row.get("doc_type", "kb")
            tenant_id = row.get("tenant_id", "global")
            tags = row.get("tags", []) or []
            redactions = int(row.get("redaction_count", 0) or 0)
            redact_note = " [has redactable lines]" if redactions > 0 else ""
            print(f"✅ {score:0.3f} | {title} | type={doc_type} tenant={tenant_id} tags={tags}{redact_note}")
    else:
        print("🚫 No visible documents in the top-K results.")

    blocked_count = len(hits) - len(visible)
    if blocked_count > 0:
        print(f"\n🔒 BLOCKED {blocked_count} of top-{len(hits)}")
        for k, v in blocked_reasons.items():
            print(f"  - {k}: {v}")

        print("\nBlocked examples (first 10):")
        shown = 0
        for i, score in hits:
            row = df.iloc[i]
            ok, reason = rbac_allow(row, user_role, user_tenant, clearance_tags)
            if ok:
                continue
            title = row.get("title", "(no title)")
            doc_type = row.get("doc_type", "kb")
            tenant_id = row.get("tenant_id", "global")
            allowed_roles = row.get("allowed_roles", []) or []
            restricted_tags = row.get("restricted_tags", []) or []
            print(f"🚫 {score:0.3f} | {reason} | {title} | type={doc_type} tenant={tenant_id} roles={allowed_roles} restricted={restricted_tags}")
            shown += 1
            if shown >= 10:
                break


# Widgets (optional)
role_dropdown = widgets.Dropdown(options=ALL_ROLES, value="analyst", description="Role:")
tenant_dropdown = widgets.Dropdown(options=["acme", "globex", "initech"], value="acme", description="Tenant:")
clearance_select = widgets.SelectMultiple(
    options=["pii", "secrets", "customer-data", "prod", "credentials"],
    value=(),
    description="Clearance:",
)
topk_slider = widgets.IntSlider(value=30, min=5, max=100, step=5, description="Top-K:")
search_box = widgets.Text(placeholder='Try: "GPU latency impacts APR", "Kubernetes incident runbook"', description='Search:')
btn_rbac = widgets.Button(description="Run Secure Semantic Search")


def on_rbac_click(b):
    clear_output(wait=True)
    display(ui_rbac)
    rbac_vector_search(
        search_box.value,
        role_dropdown.value,
        tenant_dropdown.value,
        set(clearance_select.value),
        top_k=int(topk_slider.value),
    )


btn_rbac.on_click(on_rbac_click)
ui_rbac = widgets.VBox([role_dropdown, tenant_dropdown, clearance_select, topk_slider, search_box, btn_rbac])

display(ui_rbac)

NameError: name 'embedder' is not defined

In [10]:
# ---- NO-WIDGET FALLBACK ----
# If ipywidgets don't render/click in your environment, run semantic search directly:

print("\n=== Non-widget demo (direct calls) ===")

# Security (cross-tenant) should see the most.
rbac_vector_search("Kubernetes incident runbook", "security", "acme", set(), top_k=30)

print("\n---")
# Analyst should see fewer (role-gated) and may hit restricted_tags blocks.
rbac_vector_search("GPU latency impacts APR", "analyst", "acme", set(), top_k=30)

print("\n---")
# Analyst with secrets clearance.
rbac_vector_search("SECRET", "analyst", "acme", {"secrets"}, top_k=30)




=== Non-widget demo (direct calls) ===
🔎 Semantic search: 'Kubernetes incident runbook' as role='security' tenant='acme' clearance=[]
----- Results -----
✅ 0.682 | ["...", "..."] } assistant { "title": "Kubernetes Incident Response for Crash Loops", "body": "In the event of a Kubernetes cluster crash lo | type=runbook tenant=globex tags=['docker', 'driver', 'kubernetes', 'latency', 'observability']
✅ 0.565 | ... | type=runbook tenant=initech tags=['...', 'docker', 'driver', 'firewall', 'kubernetes', 'latency']
✅ 0.543 | ... | type=runbook tenant=initech tags=['...', 'docker', 'firewall', 'gpu', 'kubernetes', 'observability']
✅ 0.530 | ... | type=runbook tenant=acme tags=['...', 'docker', 'driver', 'gpu', 'kubernetes', 'observability']
✅ 0.501 | "title": "...", "body": "...", "tags": ["...", "..."] } assistant { "title": "Driver/CUDA Compatibility and Container Deployment Issues", "b | type=runbook tenant=acme tags=['docker', 'driver', 'firewall', 'gpu', 'kubernetes']
✅ 0.483 | ... | t